In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

一共有三个作者，应该是用rnn效果最好

In [2]:
data_path = 'train.csv'
data = pd.read_csv(data_path)
data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [3]:
data_path = 'test.csv'
test_data = pd.read_csv(data_path)
test_data.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


删除停止词

In [4]:
stopwords = [ line.rstrip() for line in open('stop_words.txt') ]
stopwords[:10]

['a',
 'able',
 'about',
 'above',
 'abst',
 'across',
 'act',
 'actually',
 'added',
 'adj']

In [5]:
def clean_stop_words(words,stopwords=stopwords):
    cleaned_words = []
    for word in words:
        if word not in stopwords:
            cleaned_words.append(word)
    return cleaned_words

规整数据，做一个字典，将所有出现的文字都encoding到字典中

In [6]:
from collections import Counter
counts = Counter()
for line in data['text']:
    words = line.lower().split()
    #words = clean_stop_words(words)
    counts.update(words)
    
for line in test_data['text']:
    words = line.lower().split()
    #words = clean_stop_words(words)
    counts.update(words)

vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)}

def text_to_ints(text):#这里传进来的是一个text
    text_ints = [vocab_to_int[word] for word in text.lower().split()]
    text_ints = np.array(text_ints)
    return text_ints

In [7]:
print('vocab length:',len(vocab_to_int))
vocab_to_int

vocab length: 53248


{'the': 0,
 'of': 1,
 'and': 2,
 'to': 3,
 'a': 4,
 'i': 5,
 'in': 6,
 'was': 7,
 'that': 8,
 'my': 9,
 'had': 10,
 'with': 11,
 'it': 12,
 'he': 13,
 'his': 14,
 'as': 15,
 'for': 16,
 'which': 17,
 'but': 18,
 'at': 19,
 'not': 20,
 'from': 21,
 'by': 22,
 'is': 23,
 'on': 24,
 'this': 25,
 'be': 26,
 'were': 27,
 'have': 28,
 'me': 29,
 'her': 30,
 'all': 31,
 'we': 32,
 'or': 33,
 'an': 34,
 'no': 35,
 'you': 36,
 'one': 37,
 'so': 38,
 'they': 39,
 'when': 40,
 'been': 41,
 'upon': 42,
 'its': 43,
 'could': 44,
 'she': 45,
 'would': 46,
 'there': 47,
 'their': 48,
 'more': 49,
 'some': 50,
 'him': 51,
 'what': 52,
 'our': 53,
 'now': 54,
 'into': 55,
 'are': 56,
 'than': 57,
 'very': 58,
 'if': 59,
 'who': 60,
 'will': 61,
 'only': 62,
 'about': 63,
 'these': 64,
 'any': 65,
 'up': 66,
 'even': 67,
 'did': 68,
 'then': 69,
 'yet': 70,
 'your': 71,
 'out': 72,
 'before': 73,
 'them': 74,
 'after': 75,
 'might': 76,
 'old': 77,
 'most': 78,
 'like': 79,
 'must': 80,
 'through': 81,


这里测试一下函数的结果对不对

In [8]:
test_a = data.iloc[3,1]
test_b = text_to_ints(test_a)
print('text:',test_a)
print('text_ints:',test_b)

text: How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.
text_ints: [  111   541    23   927    15    32   197    21  1708  8426    24     0
  2966  3391 12787   790  8427 12788    22   548  9515     2 19631  6862
    31   197    15     6   578   928   184 10910     2  9516]


所以train_X就这样

In [9]:
train_X = []
text_length = []
for each in data['text']:
    train_X.append(text_to_ints(each))
    text_length.append(len(text_to_ints(each)))
text_length = np.array(text_length)
train_X = np.array(train_X)
print('train_X samples:',train_X[:5])
print('max length:',text_length.max())

train_X samples: [ array([   25, 10909,   138,  1282,    29,    35,   268,     1,  8424,
           0,  5809,     1,     9, 27735,    15,     5,    76,   166,
          43, 27736,     2,   345,     3,     0,   292,  2286,     5,
         302,   994,   120,   124,   891,     1,     0, 27737,    38,
        1348,  5016,    89,     0,  2838])
 array([  12,   86,  131,  851,    3,   29,    8,    0, 6308,   76,   26,
          4,  382, 8425])
 array([    6,    14,   152,   263,     7,     4,   926,  7538,  2073,
          21,   283,    15,    13, 19630,   116,     0,  2624,  5017,
          31,   337,     1,  1283,  2965,    13,   190,  7538,  9514,
          11,    34,   247,     1,     0,   663,   426,   493, 15414])
 array([  111,   541,    23,   927,    15,    32,   197,    21,  1708,
        8426,    24,     0,  2966,  3391, 12787,   790,  8427, 12788,
          22,   548,  9515,     2, 19631,  6862,    31,   197,    15,
           6,   578,   928,   184, 10910,     2,  9516])
 array([

In [10]:
labels = []
for each in data['author']:#列表表达式貌似不能写三分支，只好使用很笨的for循环
    if each=='EAP':
        labels.append(0)
    elif each=='HPL':
        labels.append(1)
    else:
        labels.append(2)
#labels = np.array([0 if each == 'EAP' 1 if each=='HPL' else 2 for each in labels])
int_to_author={0:'EAP',1:'HPL',2:'MWS'}
train_y = np.array(labels)
train_y = pd.get_dummies(train_y)
print('train_y samples:',train_y[:20])
print('train_y shape:',train_y.shape)

train_y samples:     0  1  2
0   1  0  0
1   0  1  0
2   1  0  0
3   0  0  1
4   0  1  0
5   0  0  1
6   1  0  0
7   1  0  0
8   1  0  0
9   0  0  1
10  0  0  1
11  1  0  0
12  0  1  0
13  0  1  0
14  1  0  0
15  0  0  1
16  1  0  0
17  0  0  1
18  1  0  0
19  0  1  0
train_y shape: (19579, 3)


从上面的数据可以看出，text中最长有861个字，为了提高效率，我们还是选取200个字的长度作为输入，多退少补

In [11]:
def trunc_seq(seq,seq_len=500):
    features = np.zeros((len(seq), seq_len), dtype=int)
    for i, row in enumerate(seq):
        features[i, -len(row):] = np.array(row)[:seq_len]
    return features
train_X = trunc_seq(train_X)
print(train_X[:10])
print(train_X.shape)

[[    0     0     0 ...,    89     0  2838]
 [    0     0     0 ...,     4   382  8425]
 [    0     0     0 ...,   426   493 15414]
 ..., 
 [    0     0     0 ...,    21     9  3552]
 [    0     0     0 ...,    46   102  2441]
 [    0     0     0 ..., 12792    16   140]]
(19579, 500)


建立RNN模型

In [12]:
lstm_size = 325
lstm_layers = 1
batch_size = 500
learning_rate = 0.00001

In [13]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [14]:
n_words

53248

这里需要先做word_to_vec

In [15]:
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

构建RNN

In [16]:
with graph.as_default():
    # basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [17]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

In [18]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 3, activation_fn=tf.nn.softmax)
    #cost = tf.losses.mean_squared_error(labels_, predictions)
    cost = tf.losses.softmax_cross_entropy(labels_,predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [19]:
# 预测test集
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    #accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# batching
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [ ]:
epochs = 15

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_X, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y,
                    keep_prob: 0.6,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/15 Iteration: 5 Train loss: 1.097
Epoch: 0/15 Iteration: 10 Train loss: 1.099
Epoch: 0/15 Iteration: 15 Train loss: 1.097


导入测试集

规整数据

In [ ]:
test_X = []
test_text_length = []
for each in test_data['text']:
    test_X.append(text_to_ints(each))
    test_text_length.append(len(text_to_ints(each)))
test_text_length = np.array(text_length)
print('test_X samples:',test_X[:5])
print('max length:',text_length.max())

In [ ]:
import numpy as np
zeros = np.array([0,0,0])
for i in range(9000-len(test_X)):
    test_X.append(zeros)

In [ ]:
test_X = np.array(test_X)
test_X = trunc_seq(test_X)
test_X.shape

In [ ]:
def test_get_batches(x, batch_size=100):
    
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

In [ ]:
len(test_X)
n_batches = len(test_X)//batch_size
state_size = len(test_X)-n_batches*batch_size
state_size

In [ ]:
predicts = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, x in enumerate(test_get_batches(test_X, batch_size), 1):
        feed = {inputs_: x,
                keep_prob: 1,
                initial_state: test_state}
        predict = sess.run(predictions, feed_dict=feed)
        predicts.append(predict)
    #n_batches = len(test_X//batch_size)
    #state_size = len(test_X)-n_batches*batch_size
    #test_state_2 = sess.run(cell.zero_state(392,tf.float32))
    #predicts.append(sess.run(predictions,feed_dict={inputs_:test_X[n_batches*batch_size:],
                                                   #keep_prob:1,
                                                   #initial_state:test_state_2}))

In [ ]:
print('batch shape:',predicts[1].shape)
print('predict length:',len(predicts))
print('predict sample:\n',predicts[1])

可以看到predicts中每一个item都是一个batch的array，这里还需要将结果处理一下，根据格式要求，要求结果为每个作者的可能性，所以不需要预测到具体某个作者

In [ ]:
result = []
for each in predicts:
    for i in range(each.shape[0]):
        prob = each[i,:]
        result.append(prob)

In [ ]:
print('result length:',len(result))
print('result sample:',result[:2])

In [ ]:
#fill_result = np.zeros((len(result),3))
#for i in range(len(result)):
    #idx = result[i].argmax()
    #fill_result[i,idx] = 1

按照要求的格式将结果整理成DataFrame

In [ ]:
result_array = np.array(result)
result_array.shape

In [ ]:
result_df = pd.DataFrame(result_array,dtype=float)
result_df.columns=['EAP','HPL','MWS']
result_df = result_df[:len(test_data['id'])]
result_df = result_df.applymap(lambda x: '%.15f' % x)
result_df.insert(0,'id',test_data['id'])
result_df.head()

下面只要将结果写入到csv文件中就可以了

In [ ]:
result_df.to_csv('submision.csv',index=False)